# Ensemble predictions

In [1]:
import os
import torch
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from ultralytics import YOLO
import timm
import torch.nn as nn
from sklearn.metrics import classification_report

# Cesty
test_images_dir = "/home/jovyan/data/lightning/LiviaMurankova/new/namnozene_meteory/najlepsie_modely/vyhodnotenie/test/images"
test_labels_dir_classification = "/home/jovyan/data/lightning/LiviaMurankova/new/namnozene_meteory/najlepsie_modely/vyhodnotenie/test/labels"

model_path_convnext = "/home/jovyan/data/lightning/LiviaMurankova/new/namnozene_meteory/ConvNext-Small/datasetv2_best_convnext_model_v65_epocha2.pth"
model_path_yolov11 = "/home/jovyan/data/lightning/LiviaMurankova/new/namnozene_meteory/najlepsie_modely/najlepsie_modely/yolov11_vacsie_obrazky.pt"

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používam zariadenie: {device}")

# Transformácie
transform = transforms.Compose([
    transforms.Resize((720, 720)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Načítanie modelu ConvNeXt-Small
convnext_model = timm.create_model("convnext_small", pretrained=False, num_classes=1).to(device)
convnext_model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(convnext_model.num_features, 1)
)
convnext_model.load_state_dict(torch.load(model_path_convnext, map_location=device))
convnext_model.eval()

def predict_convnext(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = torch.sigmoid(convnext_model(image))
        return output.item()

# Načítanie modelu YOLOv11
yolo_model_11 = YOLO(model_path_yolov11).to(device)
yolo_model_11.eval()

def predict_yolo(image_path, model, threshold=0.0):
    with torch.no_grad():
        results = model(image_path, verbose=False)
        detections = results[0].boxes
    confidences = [det[4] for det in detections.data.cpu().numpy() if det[4] > threshold]
    return max(confidences) if confidences else 0.0

# Načítanie true labelu
def get_true_label(label_path):
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            content = f.read().strip()
        return 1 if content.startswith("0") else 0
    return 0

# Predikcie
results = []
image_files = sorted([
    f for f in os.listdir(test_images_dir)
    if os.path.isfile(os.path.join(test_images_dir, f)) and f.lower().endswith(('.jpg', '.jpeg', '.png'))
])

for img_name in tqdm(image_files, desc="🔍 Spracovanie obrázkov"):
    img_path = os.path.join(test_images_dir, img_name)
    label_path = os.path.join(test_labels_dir_classification, img_name.replace(".jpg", ".txt"))

    true_label = get_true_label(label_path)
    pred_convnext = predict_convnext(img_path)
    pred_yolo11 = predict_yolo(img_path, yolo_model_11)

    results.append([img_name, true_label, pred_convnext, pred_yolo11])

# Uloženie CSV
df = pd.DataFrame(results, columns=["image_name", "true_label", "prediction_ConvNexts", "prediction_yolov11"])
output_csv = "predictions_yolov11_convnext.csv"
df.to_csv(output_csv, index=False)
print(f"\n✅ CSV súbor `{output_csv}` bol úspešne vytvorený.")

# Ensemble výpočet
weights = {
    'convnexts': 0.5424,
    'yolov11': 0.4576
}

df["ensemble_probability"] = (
    df["prediction_ConvNexts"] * weights["convnexts"] +
    df["prediction_yolov11"] * weights["yolov11"]
)

threshold = 0.441
df["ensemble_prediction"] = (df["ensemble_probability"] >= threshold).astype(int)

# Výpis metrík
print("\n✅ Váhy použité pre ensemble:")
for model, w in weights.items():
    print(f"{model}: {w:.4f}")

print("\n📊 Správa klasifikácie (ensemble):")
print(classification_report(df["true_label"], df["ensemble_prediction"], digits=4))

# Uloženie finálneho CSV
final_csv = "final_predictions_with_ensemble.csv"
df.to_csv(final_csv, index=False)
print(f"\n✅ Výsledný súbor s ensemble predikciami uložený ako `{final_csv}`.")

Používam zariadenie: cuda


🔍 Spracovanie obrázkov: 100%|██████████| 924/924 [01:48<00:00,  8.52it/s]


✅ CSV súbor `predictions_yolov11_convnext.csv` bol úspešne vytvorený.

✅ Váhy použité pre ensemble:
convnexts: 0.5424
yolov11: 0.4576

📊 Správa klasifikácie (ensemble):
              precision    recall  f1-score   support

           0     0.9589    0.9352    0.9469       324
           1     0.9655    0.9783    0.9719       600

    accuracy                         0.9632       924
   macro avg     0.9622    0.9568    0.9594       924
weighted avg     0.9631    0.9632    0.9631       924


✅ Výsledný súbor s ensemble predikciami uložený ako `final_predictions_with_ensemble.csv`.


In [4]:
import torch
import torchvision
import timm
import ultralytics
import PIL
import numpy
import argparse
import sys

print("🔍 Verzie použitých knižníc:")
print(f"torch:           {torch.__version__}")
print(f"torchvision:     {torchvision.__version__}")
print(f"timm:            {timm.__version__}")
print(f"ultralytics:     {ultralytics.__version__}")
print(f"PIL (Pillow):    {PIL.__version__}")
print(f"numpy:           {numpy.__version__}")
print(f"argparse:        built-in")
print(f"Python version:  {sys.version}")


🔍 Verzie použitých knižníc:
torch:           2.6.0+cu124
torchvision:     0.21.0+cu124
timm:            1.0.15
ultralytics:     8.3.102
PIL (Pillow):    11.1.0
numpy:           1.26.4
argparse:        built-in
Python version:  3.12.8 | packaged by conda-forge | (main, Dec  5 2024, 14:24:40) [GCC 13.3.0]


In [6]:
torch.save(convnext_model.state_dict(), "convnext_model_legacy.pth", _use_new_zipfile_serialization=False)
